In [1]:
import numpy as np 
import re 
import os 
import pandas as pd
from nltk.tokenize import RegexpTokenizer , sent_tokenize
from urllib.request import urlopen
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import requests
import urllib.request,sys,time ,requests

In [2]:
stopWordsFile =     'StopWords_Generic.txt'
positiveWordsFile = 'positive-words.txt'
nagitiveWordsFile = 'negative-words.txt'

In [3]:
input = pd.read_excel("Input.xlsx")
input

def get_article_names(urls):
    titles = []
    for i in range (len(urls)):
            title = urls[i]
            title_clean = title[title.index( "m/" ) + 2 :-1]. replace('-' , ' ')
            titles.append(title_clean)
    return titles

urls =input["URL"]
urlsTitleDF = get_article_names(urls)
urlsTitleDF

['ai in healthcare to improve patient outcomes',
 'what if the creation is taking over the creator',
 'what jobs will robots take from humans in the future',
 'will machine replace the human in the future of work',
 'will ai replace us or work with us',
 'man and machines together machines are more diligent than humans blackcoffe',
 'in future or in upcoming years humans and machines are going to work together in every field of work',
 'how neural networks can be applied in various areas in the future',
 'how machine learning will affect your business',
 'deep learning impact on areas of e learning',
 'how to protect future data and its privacy blackcoffer',
 'how machines ai automations and robo human are effective in finance and banking',
 'ai human robotics machine future planet blackcoffer thinking jobs workplace',
 'how ai will change the world blackcoffer',
 'future of work how ai has entered the workplace',
 'ai tool alexa google assistant finance banking tool future',
 'ai heal

In [4]:
url = "https://insights.blackcoffer.com/ai-in-healthcare-to-improve-patient-outcomes"

page=requests.get(url , headers={"User-Agent": "XY"})  
soup = BeautifulSoup(page.text , 'html.parser')
#get title
title = soup . find("h1",attrs = { 'class' : 'entry-title'}).get_text()

#get article text
text = soup . find(attrs = { 'class' : 'td-post-content'}).get_text()
# break into lines and remove leading and trailing space on each
lines = (line.strip() for line in text.splitlines())
# break multi-headlines into a line each
chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
# drop blank lines
text = '\n'.join(chunk for chunk in chunks if chunk)

In [5]:
# Loading positive words
with open(positiveWordsFile,'r') as posfile:
    positivewords=posfile.read().lower()
positiveWordList=positivewords.split('\n')


# Loading negative words
with open(nagitiveWordsFile ,'r' ,  encoding="ISO-8859-1") as negfile:
    negativeword=negfile.read().lower()
negativeWordList=negativeword.split('\n')

#Loading stop words dictionary for removing stop words

with open(stopWordsFile ,'r') as stop_words:
    stopWords = stop_words.read().lower()
stopWordList = stopWords.split('\n')
stopWordList[-1:] = []



display( positiveWordList[:6]  , negativeWordList[:6] , stopWordList[:6])

['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable']

['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably']

['about', 'above', 'after', 'again', 'all', 'am']

In [6]:
#tokenizeing module and filtering tokens using stop words list, removing punctuations
def tokenizer(text):
    text = text.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    filtered_words = list(filter(lambda token: token not in stopWordList, tokens))
    return filtered_words

def positive_score (text):
    posword=0
    tokenphrase = tokenizer(text)
    for word in tokenphrase :
        if word in positiveWordList:
            posword+=1
      
    retpos = posword
    return retpos 

def negative_score (text):
    negword=0
    tokenphrase = tokenizer(text)
    for word in tokenphrase :
        if word in negativeWordList : negword +=1

    retneg = negword 
    return retneg

def polarity_score (positive_score , negative_score) :
    return (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
#################################################
def total_word_count(text):
    tokens = tokenizer(text)
    return len(tokens)
#############################################
def AverageSentenceLenght (text):
    Wordcount = len(tokenizer (text))
    SentenceCount = len (sent_tokenize(text))
    if SentenceCount > 0 : Average_Sentence_Lenght = Wordcount / SentenceCount

    avg = Average_Sentence_Lenght

    return round(avg)


# Counting complex words
def complex_word_count(text):
    tokens = tokenizer(text)
    complexWord = 0
    
    for word in tokens:
        vowels=0
        if word.endswith(('es','ed')):
            pass
        else:
            for w in word:
                if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
                    vowels += 1
            if(vowels > 2):
                complexWord += 1
    return complexWord

def percentage_complex_word(text):
    tokens = tokenizer(text)
    complexWord = 0
    complex_word_percentage = 0
    
    for word in tokens:
        vowels=0
        if word.endswith(('es','ed')):
            pass
        else:
            for w in word:
                if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
                    vowels += 1
            if(vowels > 2):
                complexWord += 1
    if len(tokens) != 0:
        complex_word_percentage = complexWord/len(tokens)
    
    return complex_word_percentage

def fog_index(averageSentenceLength, percentageComplexWord):
    fogIndex = 0.4 * (averageSentenceLength + percentageComplexWord)
    return fogIndex





In [7]:
URLS = input ["URL"]
URLS
     

0      https://insights.blackcoffer.com/ai-in-healthc...
1      https://insights.blackcoffer.com/what-if-the-c...
2      https://insights.blackcoffer.com/what-jobs-wil...
3      https://insights.blackcoffer.com/will-machine-...
4      https://insights.blackcoffer.com/will-ai-repla...
                             ...                        
109    https://insights.blackcoffer.com/blockchain-fo...
110    https://insights.blackcoffer.com/the-future-of...
111    https://insights.blackcoffer.com/big-data-anal...
112    https://insights.blackcoffer.com/business-anal...
113    https://insights.blackcoffer.com/challenges-an...
Name: URL, Length: 114, dtype: object

In [8]:
corps = []
titles = []
for url in URLS:
    page = requests.get(url, headers={"User-Agent": "XY"})
    soup = BeautifulSoup(page.text, 'html.parser')

    # get title
    title = soup.find('h1', class_='entry-title')
    if title:
        title = title.get_text()

    # get article text
    text = soup.find(class_='td-post-content')
    if text:
        # break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in text.get_text().splitlines())
        # break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        # drop blank lines
        text = '\n'.join(chunk for chunk in chunks if chunk)
        corps.append(text)
        titles.append(title)


In [9]:
df = pd.DataFrame({'title':titles,'corps': corps})
df

,title,corps
0,AI in healthcare to Improve Patient Outcomes,Introduction\n“If anything kills over 10 milli...
1,What if the Creation is Taking Over the Creator?,"Human minds, a fascination in itself carrying ..."
2,What Jobs Will Robots Take From Humans in The ...,Introduction\nAI is rapidly evolving in the em...
3,Will Machine Replace The Human in the Future o...,“Anything that could give rise to smarter-than...
4,Will AI Replace Us or Work With Us?,“Machine intelligence is the last invention th...
...,...,...
106,Blockchain for Payments,Reconciling with the financial realities of an...
107,The future of Investing,What Is an Investment?\nAn investment is a res...
108,Big Data Analytics in Healthcare,Quality and affordable healthcare is a vision ...
109,Business Analytics In The Healthcare Industry,Analytics is a statistical scientific process ...


In [10]:
# df = pd.DataFrame(corps,urlsTitleDF)
# df = pd.DataFrame({'title':urlsTitleDF,'corps': corps})
df["total word count"] = df["corps"] . apply (total_word_count)
df["percentage_complex_word"] = df["corps"] . apply (percentage_complex_word)
df["complex_word_count"] = df["corps"] . apply (complex_word_count)
df["AverageSentenceLenght"] = df["corps"] . apply (AverageSentenceLenght)
df["positive_score"] = df["corps"] . apply (positive_score)
df["negative_score"] = df["corps"] . apply (negative_score)
df["polarity_score"] = np.vectorize(polarity_score)(df['positive_score'],df['negative_score'])

df

,title,corps,total word count,percentage_complex_word,complex_word_count,AverageSentenceLenght,positive_score,negative_score,polarity_score
0,AI in healthcare to Improve Patient Outcomes,Introduction\n“If anything kills over 10 milli...,1205,0.434855,524,16,82,36,0.389831
1,What if the Creation is Taking Over the Creator?,"Human minds, a fascination in itself carrying ...",830,0.295181,245,10,72,37,0.321101
2,What Jobs Will Robots Take From Humans in The ...,Introduction\nAI is rapidly evolving in the em...,1055,0.433175,457,12,74,36,0.345455
3,Will Machine Replace The Human in the Future o...,“Anything that could give rise to smarter-than...,973,0.345324,336,10,86,28,0.508772
4,Will AI Replace Us or Work With Us?,“Machine intelligence is the last invention th...,1101,0.335150,369,14,71,29,0.420000
...,...,...,...,...,...,...,...,...,...
106,Blockchain for Payments,Reconciling with the financial realities of an...,614,0.306189,188,13,29,29,0.000000
107,The future of Investing,What Is an Investment?\nAn investment is a res...,1060,0.346226,367,17,47,19,0.424242
108,Big Data Analytics in Healthcare,Quality and affordable healthcare is a vision ...,745,0.417450,311,11,32,49,-0.209877
109,Business Analytics In The Healthcare Industry,Analytics is a statistical scientific process ...,443,0.492099,218,15,39,7,0.695652


In [11]:


final = df.drop("corps" , 1)
final 


C:\Users\prajw\AppData\Local\Temp\ipykernel_6972\2804716251.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  final = df.drop("corps" , 1)


,title,total word count,percentage_complex_word,complex_word_count,AverageSentenceLenght,positive_score,negative_score,polarity_score
0,AI in healthcare to Improve Patient Outcomes,1205,0.434855,524,16,82,36,0.389831
1,What if the Creation is Taking Over the Creator?,830,0.295181,245,10,72,37,0.321101
2,What Jobs Will Robots Take From Humans in The ...,1055,0.433175,457,12,74,36,0.345455
3,Will Machine Replace The Human in the Future o...,973,0.345324,336,10,86,28,0.508772
4,Will AI Replace Us or Work With Us?,1101,0.335150,369,14,71,29,0.420000
...,...,...,...,...,...,...,...,...
106,Blockchain for Payments,614,0.306189,188,13,29,29,0.000000
107,The future of Investing,1060,0.346226,367,17,47,19,0.424242
108,Big Data Analytics in Healthcare,745,0.417450,311,11,32,49,-0.209877
109,Business Analytics In The Healthcare Industry,443,0.492099,218,15,39,7,0.695652


In [12]:
final.to_excel('Output Data Structure1.xlsx', encoding='utf-8')